In [1]:
# Libraries

import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


In [4]:
# === LOAD DATA ===
dataset = "dataset_rgb_224_binary_balanced.npz"
data = np.load(dataset, allow_pickle=True)
X_train, y_train = data["X_train"], data["y_train"]
X_val, y_val = data["X_val"], data["y_val"]
X_test, y_test = data["X_test"], data["y_test"]
label_names = data["label_names"]
num_classes = len(label_names)
img_size = (224, 224)

print(f"Loaded dataset with {len(X_train)} training, {len(X_val)} validation, and {len(X_test)} test samples.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"Classes: {label_names}")

Loaded dataset with 2501 training, 536 validation, and 536 test samples.
X_train shape: (2501, 224, 224, 3), y_train shape: (2501,)
X_val shape: (536, 224, 224, 3), y_val shape: (536,)
X_test shape: (536, 224, 224, 3), y_test shape: (536,)
Classes: ['NO_THREAT' 'THREAT']


In [5]:
# === Load your trained Keras model ===
model = tf.keras.models.load_model("mobilenetv2_best_model.h5")

# === Define a representative dataset generator ===
# This helps TF calibrate ranges for int8 quantization.
# Use some training/validation images (not all) to speed up conversion.
def representative_data_gen():
    for i in range(100):
        # Pick random samples from X_train
        img = X_train[i]
        img = np.expand_dims(img, axis=0).astype(np.float32)
        yield [img]

# === Convert to int8 TFLite ===
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optimization flag
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative dataset for proper quantization
converter.representative_dataset = representative_data_gen

# Force int8 I/O (needed for OpenMV)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Perform conversion
tflite_model = converter.convert()

# Save to file
with open("mobilenetv2_224_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite int8 model saved as mobilenetv2_224_int8.tflite")


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpdy4v4s8l\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpdy4v4s8l\assets


Saved artifact at 'C:\Users\user\AppData\Local\Temp\tmpdy4v4s8l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2143911310736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911311888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911312080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911309392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911312656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911311696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911312272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911312848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911313232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2143911310160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  21439

c:\Users\user\anaconda3\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ TFLite int8 model saved as mobilenetv2_224_int8.tflite
